# Python XDF Reader

## Initial Steps

### Import Libraries

In [ ]:
import os
import logging
import pyxdf
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
from scipy.fft import fftshift

### Import Data from XDF file

In [ ]:
#path = '/Users/admin/Documents/CurrentStudy/sub-P001/ses-S002/eeg'
#path = '/Users/admin/Desktop/sub-P001/ses-S001/eeg'
#path = '/Users/Min/Documents/CurrentStudy/sub-P001/ses-S001/eeg' # write directory towards xdf file
path = '/Users/Min/Documents/CurrentStudy/sub-P001/ses-S001/'
#path = '/Users/Min/Documents/CurrentStudy/' # write directory towards xdf file
logging.basicConfig(level=logging.DEBUG)  # Use logging.INFO to reduce output.
#fname = os.path.abspath(os.path.join(os.path.dirname(path),'eeg', 'StringTest.xdf'))
#fname = os.path.abspath(os.path.join(os.path.dirname(path),'Desktop', 'sub-P001_ses-S001_task-T1_run-001_eeg.xdf'))
filename = 'sub-P001_ses-S001_task-Default_run-001_eeg.xdf'
fname = os.path.abspath(os.path.join(os.path.dirname(path),'eeg', filename))
streams, fileheader = pyxdf.load_xdf(fname)

print("Found {} streams:".format(len(streams)))
for ix, stream in enumerate(streams):
    print("Stream {}: {} - type {} - uid {} - shape {} at {} Hz (effective {} Hz)".format(
        ix + 1, stream['info']['name'][0],
        stream['info']['type'][0],
        stream['info']['uid'][0],
        (int(stream['info']['channel_count'][0]), len(stream['time_stamps'])),
        stream['info']['nominal_srate'][0],
        stream['info']['effective_srate'])
    )
    if any(stream['time_stamps']):
        print("\tDuration: {} s".format(stream['time_stamps'][-1] - stream['time_stamps'][0]))
print("Done.")

In [ ]:
stream

### Initialize Variables

In [ ]:
select_stream = 0 # select the stream (Please look at this)
singledchan = 0 # selected single channel for the single channel plots
stream = streams[select_stream] # get selected stream
Fs = stream['info']['nominal_srate'] # get the effective sampling rate
select_channel = 1 # select one channel. This is for single plots
N = int(stream['footer']['info']['sample_count'][0]) # get number of samples
channum = int(stream['info']['channel_count'][0]) # get number of channels
data_Y = stream['time_series'] # get time series data
data_X = stream['time_stamps'] # get time stamp data
data_X = (data_X-min(data_X))

#Fs = int(Fs[0])
Fs = int(float(Fs[0]))
print("Sampling Rate:", Fs)

### Trim Data 

In [ ]:
trimming = False;
if trimming: # change to true if you do want to trim data
    startidx = 127*Fs; # start idx of trim
    endidx = 142*Fs; # end idx of trim
    trimrange = list(range(startidx,endidx)) # create list from start and end idx
    data_Y = data_Y[trimrange,:] # new data
    data_X = data_X[trimrange] # end data

In [ ]:
print(stream['time_stamps'] - min(stream['time_stamps']))

## Unfiltered Analysis 

### Plot Time-Series Unfiltered (All)

In [ ]:
fig, axs = plt.subplots(channum)
for i in range(channum):
    axs[i].plot(data_X, data_Y[:,i])
    #axs[i].set_xlim(1,5)
    
fig.suptitle('Time-Series Unfiltered', fontsize=20)
fig.set_size_inches(18.5, 10.5)
fig.savefig('test2png.png',format='jpg')
#plt.close(fig)

### Plot Time-Series Unfiltered (Single)

In [ ]:
singledchan = 2
plt.figure(figsize=(18.5, 10.5))
plt.plot(data_X, data_Y[:,singledchan])
#plt.xlim((data_X[0],data_X[0]))
plt.xlim(10,20)

### PSD Unfiltered (All)

In [ ]:
plt.figure(figsize=(20, 9));
for i in range(channum):
    plt.psd(data_Y[:,i],NFFT=round(N/10),Fs=Fs, label=i);

plt.axvline(Fs/4);
plt.title('PSD: Power Spectral Density',fontsize=20);
plt.xlabel('Frequency (Hz)',fontsize=17);
plt.ylabel('Power Spectral Density (dB/Hz)',fontsize=17);
plt.tight_layout();
#plt.xticks(np.arange(100))
plt.xlim((0,Fs/2));
plt.legend(loc='upper left')
plt.show();

### PSD Unfiltered (Single)

In [ ]:
plt.figure(figsize=(20, 9));
[Pxx, freqs] = plt.psd(data_Y[:,13],NFFT=round(N/10),Fs=Fs);
plt.title('PSD: Power Spectral Density',fontsize=20);
plt.xlabel('Frequency (Hz)',fontsize=17);
plt.ylabel('Power Spectral Density (dB/Hz)',fontsize=17);
plt.tight_layout();
#plt.xticks(np.arange(250))
plt.xlim((0,250));
plt.show();

### Spectrogram Unfiltered (Single)

In [ ]:
plt.figure(figsize=(10, 9));
freqs, time, Sxx = signal.spectrogram(data_Y[:,singledchan], Fs,return_onesided=False)
plt.pcolormesh(time, fftshift(freqs), fftshift(Sxx,axes=0), shading='gouraud')
plt.title('Spectrogram Unfiltered',fontsize=20)
plt.ylabel('Frequency [Hz]',fontsize=18)
plt.xlabel('Time [sec]',fontsize=18)
plt.ylim((-100,100))
plt.show()

## Impedance

### Impedance Calculation

In [ ]:
fdata_Y = []
b, a = signal.butter(4, [19,21], btype='bandpass', fs=Fs)
w, h = signal.freqs(b, a)
# Apply Filter
for i in range(channum):
    fdata_Y.append(signal.filtfilt(b,a,data_Y[:,i]))
    
plt.figure(figsize=(18.5, 10.5))
plt.plot(data_X, fdata_Y[0])
plt.xlim([0.001, 0.002])
#plt.xlim((data_X[0],data_X[0]))

### PSD Filtered for Impedance (All)

In [ ]:
plt.figure(figsize=(10, 9));
for i in range(channum):
    plt.psd(fdata_Y[i],NFFT=round(N/10),Fs=Fs);

plt.title('PSD: Power Spectral Density',fontsize=20);
plt.xlabel('Frequency (Hz)',fontsize=17);
plt.ylabel('Power Spectral Density (dB/Hz)',fontsize=17);
plt.tight_layout();
plt.xlim((0,100));
plt.show();

### Finds Which Channels are Railed

In [ ]:
idx = [i for i,v in enumerate(freqs) if v > 50 and v < 100]
for i in range(channum):
    [Pxx, freqs] = plt.psd(data_Y[:,i],NFFT=round(N/10),Fs=Fs);
    if np.mean(10*np.log10(Pxx[idx])) < -16:
        print(i)
#print(Pxx)
#0,3,5,6,7,8,9,10,11,15

## Filtered

### Apply Filter

In [ ]:
# Initialize Variables
F0 = 60.0  # Frequency to be removed from signal (Hz)
Q = 30.0  # Quality factor
data_Y = stream['time_series'] # get time series data
data_X = stream['time_stamps'] # get time stamp data
data_X = (data_X-min(data_X))

# Design notch filter
b, a = signal.iirnotch(F0, Q, Fs)
fdata_Y = []

# Apply Filter
for i in range(channum):
    fdata_Y.append(signal.filtfilt(b,a,data_Y[:,i]))

fdata_Y = np.array(fdata_Y)

### Plot Time-Series Filtered (All)

In [ ]:
fig, axs = plt.subplots(channum) # define subplots

for i in range(channum):
    axs[i].plot(new_data_X, new_fdata_Y[i])
    
fig.suptitle('Time-Series Filtered', fontsize=20)
fig.set_size_inches(18.5, 10.5)
fig.savefig('test2png.png',format='jpg')
#plt.close(fig)

### PSD Filtered (All)

In [ ]:
plt.figure(figsize=(10, 9));
for i in range(channum):
    plt.psd(new_fdata_Y[i],NFFT=round(N/10),Fs=Fs);

plt.title('PSD: Power Spectral Density',fontsize=20);
plt.xlabel('Frequency (Hz)',fontsize=17);
plt.ylabel('Power Spectral Density (dB/Hz)',fontsize=17);
plt.tight_layout();
plt.xlim((0,100));
plt.show();

### Spectrogram Filtered (Single)

In [ ]:
plt.figure(figsize=(10, 9));
freqs, time, Sxx = signal.spectrogram(new_fdata_Y[singledchan], Fs,return_onesided=False)
plt.pcolormesh(time, fftshift(freqs), fftshift(Sxx,axes=0), shading='gouraud')
plt.title('Spectrogram Unfiltered',fontsize=20)
plt.ylabel('Frequency [Hz]',fontsize=18)
plt.xlabel('Time [sec]',fontsize=18)
plt.ylim((-100,100))
plt.show()